In [18]:
library(dplyr)
library(sf)

Linking to GEOS 3.9.1, GDAL 3.3.2, PROJ 8.1.1



In [7]:
chinese_wild_life_list <- read.csv('taxonomy.csv') %>% 
                        select('scientificName','kingdomName','phylumName','className','orderName','familyName')
Chinese_species_distribution_grid <- read.csv("../species_distribution_grid_filtered.csv",
                                              header = F, col.names = c("class","species","grid_id","area")) %>% 
                                    group_by(class,species,grid_id) %>% summarise(area = sum(area)) %>% ungroup()
Chinese_species_distribution_grid <- left_join(Chinese_species_distribution_grid,chinese_wild_life_list,by = c('species' = 'scientificName'))
Chinese_species_distribution_grid_selected <- filter(Chinese_species_distribution_grid,
                                                     !(familyName %in% toupper(c('Procellariidae','Alcidae','Fregatidae','Gaviidae',
                                                                       'Phaethontidae','Diomedeidae','Stercorariidae','Sulidae',
                                                                       'Pelecanidae','Phalacrocoracidae','Laridae','Hydrobatidae', # marine bird
                                                                       'Balaenopteridae','Delphinidae','Eschrichtiidae','Ziphiidae',
                                                                       'Kogiidae','Dugongidae','Iniidae','Phocoenidae','Physeteridae',
                                                                       'Phocidae','Otariidae','Sirenia')))) %>%  # marine mammal
                                                dplyr::select('className','orderName','familyName','species') %>% as.data.frame() 


`summarise()` has grouped output by 'class', 'species'. You can override using the `.groups` argument.



In [8]:
Chinese_species_distribution_grid_selected$species <- gsub(' ','_',Chinese_species_distribution_grid_selected$species)

In [12]:
Chinese_species_distribution_grid_selected <- Chinese_species_distribution_grid_selected[!duplicated(Chinese_species_distribution_grid_selected),]
head(Chinese_species_distribution_grid_selected)
write.table(Chinese_species_distribution_grid_selected,'taxonomy_.csv',sep = '_',row.names = F,quote = F,col.names = F)

,className,orderName,familyName,species
,<chr>,<chr>,<chr>,<chr>
1,AMPHIBIA,ANURA,RANIDAE,Amolops_albispinus
7,AMPHIBIA,ANURA,RANIDAE,Amolops_aniqiaoensis
11,AMPHIBIA,ANURA,RANIDAE,Amolops_bellulus
13,AMPHIBIA,ANURA,RANIDAE,Amolops_chayuensis
14,AMPHIBIA,ANURA,RANIDAE,Amolops_chunganensis
152,AMPHIBIA,ANURA,RANIDAE,Amolops_daiyunensis


In [33]:
# add lizards in Chinese213lizards_9_27 BUT NOT IN Chinese_species_distribution_grid_selected
lizard <- sf::st_read('../../reptile/data_reptile/Chinese213lizards_9_27/Chinese_lizards_214_version20200927.shp') %>% st_set_geometry(NULL)
lizard$Binomial <- gsub(' ','_',lizard$Binomial)
lizard <- within(lizard,{
    Group_ <- dplyr::recode(Group_,
                  'Agamid' = 'Agamidae',
                  'Scincid' = 'Scincidae',
                  'Dibamid' = 'Dibamidae',
                  'Lacertid' = 'Lacertidae',
                  'Shinisaurid' = 'Shinisauridae ',
                  'Varanid' = 'Varanidae' )
})
taxa <- filter(lizard,!(Binomial %in% Chinese_species_distribution_grid_selected$species)) %>% select(Group_,Binomial) %>% 
    mutate(taxa = paste0('REPTILIA_SQUAMATA_',Group_,'_',Binomial)) %>% select(taxa)
write.table(taxa,'./lizard_sup.txt',sep = '\t',row.names = F,quote = F,col.names = F)

Reading layer `Chinese_lizards_214_version20200927' from data source 
  `/home/hmp/paleoclimate/reptile/data_reptile/Chinese213lizards_9_27/Chinese_lizards_214_version20200927.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 213 features and 11 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: 73.44696 ymin: 18.1615 xmax: 135.0858 ymax: 53.55793
Geodetic CRS:  WGS 84


In [22]:
head(lizard)
unique(lizard$Group_)

,OBJECTID_1,Binomial,Area,Group_,Shape_Leng,Shape_Le_1,物种,科,Shape_Le_2,Shape_Area,logBM
,<int>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>
1,1,Acanthosaura armata,217537.2,Agamid,11.16604,11.16604,长棘蜥,鬣蜥科,11.16604,2.721448,55.87
2,2,Acanthosaura lepidogaster,2557954.5,Agamid,134.73287,134.73287,丽棘蜥,鬣蜥科,134.73287,77.734838,41.23
3,3,Alsophylax przewalskii,863889.3,Gecko,45.70738,45.70738,新疆漠虎,壁虎科,45.70738,90.565280,0.819
4,4,Altiphylax stoliczkai,137019.6,Gecko,13.88176,13.88176,宽斑隆趾虎,壁虎科,13.88176,6.001921,2.810
5,5,Ablepharus alaicus,476152.1,Scincid,47.08095,47.08095,阿赖山泛蜥,石龙子科,47.08095,16.679666,5.303
6,6,Scincella himalayanus,609709.2,Scincid,42.45889,42.45889,喜山滑蜥,石龙子科,42.45889,12.775777,2.370


[1] "Agamid"      "Gecko"       "Scincid"     "Dibamid"     "Lacertid"   
[6] "Shinisaurid" "Varanid"